In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.preprocessing import MinMaxScaler,LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn import metrics#参数，roc和auc
from sklearn.preprocessing import label_binarize#二值化
import joblib
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
names = ["AmismatchRate","A_Qrate","UmismatchRate","U_Qrate","GmismatchRate","G_Qrate","label"]#df.columns
path1="./new.paper-unlabled.csv"
path2="./new.treat.csv"
# path1="~/project/t3_ontnewrna/output/paper-unlabled/TrainData/CrandomForestTrainData.csv"
# path2="~/project/t3_ontnewrna/output/OV-FTO-2-0603/TrainData/TrandomForestTrainData.csv"
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df1["label"]=0
df2["label"]=1
df=pd.concat([df1,df2])
df = df[df["coverage"]>=0.8]
df = df[names]
df=df.replace([np.inf, -np.inf], np.nan)
# df
# df.eval("label = label1",inplace=True)
# df=df.drop("label0",axis=1)
print(df.shape)
df

(1628237, 7)


,AmismatchRate,A_Qrate,UmismatchRate,U_Qrate,GmismatchRate,G_Qrate,label
7,0.0125,0.0550,0.0063,0.0238,0.0047,0.0099,0
8,0.0033,0.0181,0.0000,0.0000,0.0166,0.0397,0
9,0.0015,0.0041,0.0167,0.0761,0.0091,0.0231,0
10,0.0062,0.0212,0.0062,0.0257,0.0109,0.0284,0
12,0.0163,0.0746,0.0245,0.1150,0.0245,0.0651,0
...,...,...,...,...,...,...,...
888632,0.0347,0.0949,0.0146,0.0586,0.0037,0.0224,1
888639,0.0202,0.1093,0.0289,0.1585,0.0190,0.0420,1
888640,0.0136,0.0592,0.0195,0.0946,0.0161,0.0362,1
888653,0.0246,0.0965,0.0320,0.1712,0.0269,0.0771,1


In [3]:
#.划分测试集合训练集

x=df.iloc[:,0:-1]
y=df.iloc[:,-1:]

x=x.replace("inf", np.NAN)

#通过平局值来替换nan
#imputer = SimpleImputer(missing_values="NaN")
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
x=imputer.fit_transform(x,y)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
print("训练样本数量:%d,特征属性数目:%d,目标属性数目:%d"%(x_train.shape[0],x_train.shape[1],y_train.shape[1]))

训练样本数量:1302589,特征属性数目:6,目标属性数目:1


In [4]:
# ss = MinMaxScaler()
#x_train
# y_train
ss = MinMaxScaler()
x_train2 = x_train
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)
x_train2 = ss.transform(x_train2)
y12 = []
for i in y_test["label"]:
    y12.append(i)

In [5]:
rf = RandomForestClassifier(n_estimators=100,criterion="gini",max_depth=30,oob_score=True,random_state=1)
rf.fit(x_train,y_train)
#joblib.dump(rf, "test.pkl")
score = rf.score(x_test,y_test)
score2 = rf.score(x_train2,y_train)
#score2 = rf.socre(x_train,y_train)
print("准确率：%.2f%%"%(score*100))
print("准确率：%.2f%%"%(score2*100))
# cv_results = cross_validate(rf,x_train,y_train,return_train_score=True)
# print(cv_results["train_score"].mean())
# print(cv_results["test_score"].mean())
# forest_y_score1 = rf.predict(x_test)
# forest_y_score1

C:\Users\panda\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


MemoryError: could not allocate 29360128 bytes

In [9]:
feat_labels = df.columns[0:-1]
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(x_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 6, feat_labels[indices[f]], importances[indices[f]]))

 1) UmismatchRate 0.534588
 2) U_Qrate  0.278945
 3) AmismatchRate 0.064735
 4) GmismatchRate 0.035537
 5) C_Qrate  0.027483
 6) A_Qrate  0.023224
 7) G_Qrate  0.021300
 8) CmismatchRate 0.014188
